# Liage entre dicotopo et le pouillé

In [ ]:
#On importe nos dataframes nécessaires à la réinjection

import pandas as pd
import re
import pyexcel_ods
from unidecode import unidecode

#On importe les différents fichiers nécessaires. 

place_old_label = pd.read_csv('../../utils/pouille/resources/place_old_label.tsv', sep='\t', dtype=str)
places = pd.read_csv('../../utils/pouille/resources/place.tsv', sep='\t', dtype=str)
liage_po7 = pd.read_csv('../../utils/pouille/out/linking_out/liage_po7.csv', dtype=str)

In [ ]:
#On applique la fonction voulue par Olivier pour supprimer les accents etc

def replace_special_characters(text):
    text = unidecode(text)
    text = re.sub(r'[-\']', ' ', text)
    return text

liage_po7['vedette'] = liage_po7['vedette'].astype(str)
places['label'] = places['label'].astype(str)

liage_po7['vedette'] = liage_po7['vedette'].apply(replace_special_characters)
places['label'] = places['label'].apply(replace_special_characters)

In [ ]:
# On ne garde que les départements utiles à notre liage

dpt_list = liage_po7['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)  

In [ ]:
# dataframe à charger
liages_df = pd.DataFrame()

In [ ]:
columns_to_drop_po = ['localisationde', 'dpt_code', 'canton_code', 'method', 'reference']

In [ ]:
columns_to_drop_places = ['dpt', 'country', 'localization_commune_relation_type', 'responsibility_id']

In [ ]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = liage_po7[liage_po7['localisationco'] == 'none']
communes_df = communes_df.drop(columns=columns_to_drop_po)
places = places.drop(columns=columns_to_drop_places)

# Communes exact match

In [ ]:
#on lie par exact match vedette / label

#on enlève les lignes sans insee_code de liage_po7: elles ne servent à rien
communes_df_nonan = communes_df.dropna(subset=['insee_code'])

liage_exact_communes = pd.merge(communes_df_nonan,
                      places,
                      how='inner',
                      left_on=['insee_code','vedette'],
                      right_on=['commune_insee_code', 'label'])


# Check for duplicate entries
duplicates = liage_exact_communes.duplicated(['article_id'], keep=False)
if duplicates.any():
    liage_exact_communes = liage_exact_communes[~duplicates]
    
# Add a 'method' column with value 'dpt_exact'
liage_exact_communes['method_dicotopo'] = 'communes_exact'


# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_exact_communes]).drop_duplicates()

In [ ]:
liages_df

# Lieux dans des communes exact match

In [ ]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

localisationco_df = liage_po7[liage_po7['localisationco'] != 'none']
localisationco_df = localisationco_df.drop(columns=columns_to_drop_po)

In [ ]:
#on lie par exact match vedette de la localisationco / label

#on enlève les lignes sans insee_code de liage_po7: elles ne servent à rien
localisationco_df_nonan = localisationco_df.dropna(subset=['insee_code'])

liage_exact_localisationco = pd.merge(localisationco_df_nonan,
                      places,
                      how='inner',
                      left_on=['insee_code','vedette'],
                      right_on=['localization_commune_insee_code', 'label'])


# Check for duplicate entries
duplicates = liage_exact_localisationco.duplicated(['article_id'], keep=False)
if duplicates.any():
    liage_exact_localisationco = liage_exact_localisationco[~duplicates]
    
# Add a 'method' column with value 'dpt_exact'
liage_exact_localisationco['method_dicotopo'] = 'localisationco_exact'


# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_exact_localisationco]).drop_duplicates()

In [ ]:
liages_df.agg(['nunique', 'count', 'size'])

# Fuzzy match communes

In [ ]:
import pandas as pd
import difflib

communes_df_nonan = communes_df_nonan[~communes_df_nonan.article_id.isin(liages_df['article_id'])]

liage_fuzzy_communes = pd.merge(communes_df_nonan,
                                places,
                                how='inner',
                                left_on=['insee_code'],
                                right_on=['commune_insee_code'])


def partial_match(row):
    vedette = row['vedette']
    label = row['label']

    if isinstance(vedette, str) and not pd.isnull(vedette) and isinstance(label, str) and not pd.isnull(label):
        words = vedette.split()
        if any(difflib.get_close_matches(word, label.split(), n=1) for word in words):
            return True

    return False

liage_fuzzy_communes = liage_fuzzy_communes[liage_fuzzy_communes.apply(partial_match, axis=1)]

# Add a 'method' column with value 'dicotopo_fuzzy'
liage_fuzzy_communes['method_dicotopo'] = 'communes_fuzzy'

# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_fuzzy_communes]).drop_duplicates()

In [ ]:
liage_fuzzy_communes.agg(['nunique', 'count', 'size'])

# Fuzzy match localisationco

In [ ]:
import pandas as pd
import difflib

localisationco_df_nonan = localisationco_df_nonan[~localisationco_df_nonan.article_id.isin(liages_df['article_id'])]

liage_fuzzy_localisationco = pd.merge(localisationco_df_nonan,
                                places,
                                how='inner',
                                left_on=['insee_code'],
                                right_on=['localization_commune_insee_code'])


def partial_match(row):
    vedette = row['vedette']
    label = row['label']

    if isinstance(vedette, str) and not pd.isnull(vedette) and isinstance(label, str) and not pd.isnull(label):
        words = vedette.split()
        if any(difflib.get_close_matches(word, label.split(), n=1) for word in words):
            return True

    return False

liage_fuzzy_localisationco = liage_fuzzy_localisationco[liage_fuzzy_localisationco.apply(partial_match, axis=1)]

# Add a 'method' column with value 'dicotopo_fuzzy'
liage_fuzzy_localisationco['method_dicotopo'] = 'localisationco_fuzzy'

# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_fuzzy_localisationco]).drop_duplicates()

In [ ]:
liage_fuzzy_localisationco.agg(['nunique', 'count', 'size'])

# Exportation

In [ ]:
#on append les deux dataframes ensemble

new_rows_df = liage_po7[~liage_po7['article_id'].isin(liages_df['article_id'])]

if 'method_dicotopo' not in new_rows_df.columns:
    new_rows_df['method_dicotopo'] = 'nulle'

liages_df = liages_df.append(new_rows_df, ignore_index=True)

In [ ]:
liages_df.agg(['nunique', 'count', 'size'])

In [ ]:
#on ajoute les lignes de notre pouillé qui n'ont pas eu de match avec dicotopo pour vérification manuelle

remaining_rows = liage_po7[~liage_po7['article_id'].isin(liages_df['article_id'])]
updated_liages_df = pd.concat([liages_df, remaining_rows], ignore_index=True)

#on sort par article_id pour plus de clarté
updated_liages_df = updated_liages_df.sort_values('article_id')
updated_liages_df = updated_liages_df.reset_index(drop=True)

In [ ]:
updated_liages_df

In [ ]:
updated_liages_df.agg(['nunique', 'count', 'size'])

In [ ]:
# Sauvegarder le dataframe fusionné
updated_liages_df.to_csv('../../utils/pouille/out/reinjection_out/po7_dicotopo.csv', index=False)

In [ ]:
#partie pour plus tard

# Réinsertion dans le fichier XML

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

# Charger le fichier XML
tree = ET.parse('../../utils/pouille/out/linking_out/PO_t7_modified.xml')
root = tree.getroot()
# Replace nan values with empty string in the DataFrame (juste pour le temps où olivier corrige)
liages_df = liages_df.replace({np.nan: ""})

In [ ]:
# Pour chaque article dans le XML
for article in root.findall('article'):
    # Récupérer l'article_id
    article_id = article.get('old-id')

    # Vérifier si l'article_id est présent dans le DataFrame
    if article_id in liages_df['article_id'].values:
        # Récupérer la ligne correspondante dans le DataFrame
        liage_row = liages_df.loc[liages_df['article_id'] == article_id]

        # Ajouter la balise <dicotopo> à la balise article avec le code place_id
        dicotopo_code = liage_row['place_id'].values[0]
        dicotopo_elem = ET.Element('dicotopo')
        dicotopo_elem.text = dicotopo_code
        article.append(dicotopo_elem)

In [ ]:
# Enregistrer le fichier XML modifié
tree.write('../../utils/pouille/out/reinjection_out/po7_dicotopo.xml', encoding='UTF-8', xml_declaration=True)